## LSTM for Gait Phase Estimation

The purpose of this notebook is to train LSTM models to estimate a user's gait phase on both legs, independent of the locomotion mode.

In [1]:
from lstm_modules import *
from convolutional_nn import *
from data_processing import *
import numpy as np
import tensorflow as tf
import itertools
import pandas as pd

Using TensorFlow backend.


In [2]:
# Seed Random Number Generators

### Set Up Data

In [3]:
sensors = ['leftJointPosition', 'rightJointPosition', 'leftJointVelocity',
           'rightJointVelocity', 'imuGyroX', 'imuGyroY', 'imuGyroZ', 'imuAccX',
           'imuAccY', 'imuAccZ']

# Produce 1 label file for each trial and store them in ../labels folder
data = import_data(sensors)

left_joint_positions, right_joint_positions = extract_joint_positions(data)
labels = []
for i in range(5):
    filename = "labels/label_trial{}.txt".format(i+1)
    left_gp_x, left_gp_y = label_vectors(left_joint_positions[i])
    right_gp_x, right_gp_y = label_vectors(right_joint_positions[i])
    labeled_gp = pd.DataFrame({'leftGaitPhase_x': left_gp_x, 'leftGaitPhase_y': left_gp_y,
                               'rightGaitPhase_x': right_gp_x, 'rightGaitPhase_y': right_gp_y})
    labels.append(labeled_gp)

# Combine the data and the labels
for d, l in zip(data, labels):
    d[l.columns] = l

# Creat a list of cut_indicies for each trial
cut_indicies_list = []
for i in range(5):
    cut_indicies_list.append(find_cutting_indices(left_joint_positions[i], 
    right_joint_positions[i]))

# Cut the standing data and store files into ../features folder
data_list = cnn_cut_data(data, cut_indicies_list)

### Tune Hyperparameters

In [4]:
hyperparam_space = {
    'window_size': [120],
    'lstm': {
      'units': [10],
      'activation': ['relu'],
      'recurrent_regularizer': ['l2'],
      'kernel_regularizer': ['l2']
    },
    'dense': {
        'activation': ['tanh'],
        'kernel_regularizer': ['l2']
    },
    'optimizer': {
        'loss': ['mean_absolute_error'],
        'optimizer': ['adam']
    }
}

In [5]:
def get_model_configs(hyperparam_space):
    model_configs = []
    for window_size in hyperparam_space['window_size']:
        lstm_params = list(hyperparam_space['lstm'].keys())

        lstm_possibilities = []

        for param in lstm_params:
            lstm_possibilities.append(hyperparam_space['lstm'][param])

        lstm_config_tuples = itertools.product(*lstm_possibilities)
        lstm_configs = []
        for config in lstm_config_tuples:
            lstm_config = {}
            for i, value in enumerate(config):
                lstm_config[lstm_params[i]] = value
            lstm_configs.append(lstm_config)

        dense_params = list(hyperparam_space['dense'].keys())

        dense_possibilities = []

        for param in dense_params:
            dense_possibilities.append(hyperparam_space['dense'][param])

        dense_config_tuples = itertools.product(*dense_possibilities)
        dense_configs = []
        for config in dense_config_tuples:
            dense_config = {}
            for i, value in enumerate(config):
                dense_config[dense_params[i]] = value
            dense_configs.append(dense_config)

        optim_params = list(hyperparam_space['optimizer'].keys())

        optim_possibilities = []

        for param in optim_params:
            optim_possibilities.append(hyperparam_space['optimizer'][param])

        optim_config_tuples = itertools.product(*optim_possibilities)
        optim_configs = []
        for config in optim_config_tuples:
            optim_config = {}
            for i, value in enumerate(config):
                optim_config[optim_params[i]] = value
            optim_configs.append(optim_config)

        possible_configs = itertools.product(lstm_configs, dense_configs, optim_configs)
        config_count = 0
        for config in possible_configs:
            config_count += 1
            model_configs.append({
                'window_size': window_size,
                'lstm': config[0],
                'dense': config[1],
                'optimizer': config[2]
            })
    return model_configs            

In [6]:
hyperparameter_configs = get_model_configs(hyperparam_space)
print(len(hyperparameter_configs))

1


In [7]:
results = []
for model_config in hyperparameter_configs:
    current_result = {}
    current_result['model_config'] = model_config
    current_result['left_validation_rmse'] = []
    current_result['right_validation_rmse'] = []
    for trial in np.arange(1,11):
        dataset = cnn_extract_features(data_list, model_config['window_size'], trial)
#         dataset = cnn_train_test_split(trial, model_config['window_size'])
        model = lstm_model(sequence_length=model_config['window_size'],
                          n_features=10, 
                           lstm_config=model_config['lstm'],
                           dense_config=model_config['dense'],
                           optim_config=model_config['optimizer'],
                           X_train=dataset['X_train'].squeeze())
        model.summary()
        early_stopping_callback = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0)
        model_hist = model.fit(dataset['X_train'].squeeze(), dataset['y_train'].squeeze(), epochs=30, batch_size=128, verbose=1, validation_split=0.2, shuffle=True, callbacks= [early_stopping_callback])
        # model.save('test_rnn_model')

        predictions = model.predict(dataset['X_test'].squeeze())
        left_rmse, right_rmse = custom_rmse(dataset['y_test'].squeeze(), predictions)

        current_result['left_validation_rmse'].append(left_rmse)
        current_result['right_validation_rmse'].append(right_rmse)
    results.append(current_result)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization (Normalization (None, 120, 10)           21        
_________________________________________________________________
lstm (LSTM)                  (None, 10)                840       
_________________________________________________________________
dense (Dense)                (None, 4)                 44        
Total params: 905
Trainable params: 884
Non-trainable params: 21
_________________________________________________________________
Train on 27496 samples, validate on 6875 samples
Epoch 1/30
27496/27496 [==============================] - 17s 601us/sample - loss: 1.0741 - val_loss: 0.9868
Epoch 2/30
27496/27496 [==============================] - 17s 608us/sample - loss: 0.8573 - val_loss: 0.7553
Epoch 3/30
27496/27496 [==============================] - 19s 706us/sample - loss: 0.6552 - val_loss: 0.5925
Epoch 4/30


In [8]:
for model in results:
    model['left_rmse_mean'] = np.mean(model['left_validation_rmse'])
    model['right_rmse_mean'] = np.mean(model['right_validation_rmse'])

In [9]:
def mapper(x):
    out = {}
    out['window_size'] = x['model_config']['window_size']
    for key in x['model_config']['lstm'].keys():
        out['lstm_{}'.format(key)] = x['model_config']['lstm'][key]

    for key in x['model_config']['dense'].keys():
        out['dense_{}'.format(key)] = x['model_config']['dense'][key]

    for key in x['model_config']['optimizer'].keys():
        out['optim_{}'.format(key)] = x['model_config']['optimizer'][key]

    out['left_rmse_mean'] = x['left_rmse_mean']
    out['right_rmse_mean'] = x['right_rmse_mean']
    return out
  
input_df = list(map(mapper, results))

In [10]:
print(len(results))
df_results = pd.DataFrame(input_df)
display(df_results)

1


,window_size,lstm_units,lstm_activation,lstm_recurrent_regularizer,lstm_kernel_regularizer,dense_activation,dense_kernel_regularizer,optim_loss,optim_optimizer,left_rmse_mean,right_rmse_mean
0,120,10,relu,l2,l2,tanh,l2,mean_absolute_error,adam,2.655973,2.720643


In [11]:
df_results.to_csv('results_2.csv')

In [ ]:
# print(left_rmse)
# print(right_rmse)
# plt.figure(1)
# plt.plot(gp['left_true'])
# plt.plot(gp['left_pred'])
# plt.legend(['GT', 'Pred'])
# plt.show()

# Define list of hyperparameter options
# For each hyperparameter combination
#   Create model using hyperparameters
#   Train model
#   Store hyperparameters and validation loss
#   Store plot of training/validation loss over time

In [ ]:
# n_epochs = len(model_hist.history['loss'])
# epochs = np.arange(1,n_epochs+1)
# plt.figure(1)
# plt.plot(epochs, model_hist.history['loss'])
# plt.plot(epochs, model_hist.history['val_loss'])
# plt.legend(['Training Loss', 'Validation Loss'])
# plt.show()